<a href="https://colab.research.google.com/github/Sidhtang/ivnsto-assignment/blob/main/stock_prediciton_using_lstm_and_attention_mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import datetime as dt

print(tf.__version__)

2.15.0


In [2]:
# fetch APPL data
aapl_data = yf.download('AAPL', start='2020-01-01', end='2024-1-1')
aapl_data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.960442,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.251144,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.826851,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.484352,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.650352,132079200


In [3]:
# Checking for missing values
aapl_data.isnull().sum()

# Filling missing values, if any
aapl_data.fillna(method='ffill', inplace=True)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
aapl_data_scaled=scaler.fit_transform(np.array(aapl_data['Close']).reshape(-1,1))


In [5]:
X= []
y=[]
for i in range(60,len(aapl_data_scaled)):
    X.append(aapl_data_scaled[i-60:i,0])
    y.append(aapl_data_scaled[i,0])

In [6]:
#train test split
train_size=int(len(X)*0.8)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [7]:
#reshapingg the data for lstm
X_train,y_train=np.array(X_train),np.array(y_train)
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,AdditiveAttention,Permute,Reshape,Multiply
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50,return_sequences=True))


In [8]:
#adding attention mechanism to the lstm
attention = AdditiveAttention(name='attention_weight')
# Permute and reshape for compatibility
model.add(Permute((2, 1)))
model.add(Reshape((-1, X_train.shape[1])))
attention_result = attention([model.output, model.output])
multiply_layer = Multiply()([model.output, attention_result])
# Return to original shape
model.add(Permute((2, 1)))
model.add(Reshape((-1, 50)))

# Adding a Flatten layer before the final Dense layer
model.add(tf.keras.layers.Flatten())

# Final Dense layer
model.add(Dense(1))

# Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
# history = model.fit(X_train, y_train, epochs=100, batch_size=25, valid


In [9]:
#optimiztion of the model
from keras.layers import BatchNormalization
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 60, 50)            20200     
                                                                 
 permute (Permute)           (None, 50, 60)            0         
                                                                 
 reshape (Reshape)           (None, 50, 60)            0         
                                                                 
 permute_1 (Permute)         (None, 60, 50)            0         
                                                                 
 reshape_1 (Reshape)         (None, 60, 50)            0         
                                                                 
 flatten (Flatten)           (None, 3000)              0

In [10]:
#training the model
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

Epoch 1/100
25/25 [==============================] - 8s 46ms/step - loss: 0.8358 - val_loss: 0.3475
Epoch 2/100
25/25 [==============================] - 0s 11ms/step - loss: 0.3358 - val_loss: 0.3876
Epoch 3/100
25/25 [==============================] - 0s 10ms/step - loss: 0.2520 - val_loss: 0.3498
Epoch 4/100
25/25 [==============================] - 0s 10ms/step - loss: 0.2238 - val_loss: 0.3137
Epoch 5/100
25/25 [==============================] - 0s 10ms/step - loss: 0.2024 - val_loss: 0.2945
Epoch 6/100
25/25 [==============================] - 0s 13ms/step - loss: 0.1884 - val_loss: 0.2586
Epoch 7/100
25/25 [==============================] - 0s 12ms/step - loss: 0.1665 - val_loss: 0.2361
Epoch 8/100
25/25 [==============================] - 0s 12ms/step - loss: 0.1534 - val_loss: 0.2340
Epoch 9/100
25/25 [==============================] - 0s 13ms/step - loss: 0.1396 - val_loss: 0.2065
Epoch 10/100
25/25 [==============================] - 0s 10ms/step - loss: 0.1217 - val_loss: 0.1785

In [11]:
# to avoid overfitting
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
25/25 [==============================] - 0s 14ms/step - loss: 0.0108 - val_loss: 0.0025
Epoch 2/100
25/25 [==============================] - 0s 12ms/step - loss: 0.0096 - val_loss: 0.0094
Epoch 3/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0122 - val_loss: 0.0019
Epoch 4/100
25/25 [==============================] - 0s 12ms/step - loss: 0.0092 - val_loss: 0.0018
Epoch 5/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0088 - val_loss: 0.0019
Epoch 6/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0104 - val_loss: 0.0014
Epoch 7/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0092 - val_loss: 0.0149
Epoch 8/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0140 - val_loss: 0.0064
Epoch 9/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0103 - val_loss: 0.0010
Epoch 10/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0103 - val_loss: 0.0075

In [14]:
import yfinance as yf

ticker_symbol = 'AAPL'  # Replace with your desired ticker symbol
stock_data = yf.Ticker(ticker_symbol)

# Fetch the stock info
stock_info = stock_data.info

print(stock_info)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

In [15]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fetching the latest 60 days of AAPL stock data
try:
    data = yf.download('AAPL', period='60d', interval='1d')
except Exception as e:
    print(f"Error: Failed to fetch data from yfinance. {str(e)}")
    exit()

if data.empty:
    print("Error: Failed to fetch data from yfinance. Check your internet connection and ticker symbol.")
else:
    # Selecting the 'Close' price and converting to numpy array
    closing_prices = data['Close'].values

    # Scaling the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(closing_prices.reshape(-1,1))

    # Since we need the last 60 days to predict the next day, we reshape the data accordingly
    X_latest = np.array([scaled_data[-60:].reshape(60)]) # Now this should have 60 data points

    # Reshaping the data for the model (adding batch dimension)
    X_latest = np.reshape(X_latest, (X_latest.shape[0], X_latest.shape[1], 1))

    # Making predictions for the next day
    try:
        predicted_stock_price = model.predict(X_latest) # Assign the result of the prediction
        predicted_stock_price = scaler.inverse_transform(predicted_stock_price)
    except Exception as e:
        print(f"Error: Failed to make prediction. {str(e)}")
    else:
        print("Predicted Stock Price for tomorrow: ", predicted_stock_price) # Print the prediction if it exists

[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFInvalidPeriodError("%ticker%: Period '60d' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']")


Error: Failed to fetch data from yfinance. Check your internet connection and ticker symbol.
